In [ ]:
data_path = '../dataset'

In [2]:
import os

In [3]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

../dataset/test
../dataset/train


In [4]:
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

In [5]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

{0: 'chicken_curry',
 1: 'chicken_wings',
 2: 'fried_rice',
 3: 'grilled_salmon',
 4: 'hamburger',
 5: 'ice_cream',
 6: 'pizza',
 7: 'ramen',
 8: 'steak',
 9: 'sushi'}

In [6]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

{0: 'chicken_curry',
 1: 'chicken_wings',
 2: 'fried_rice',
 3: 'grilled_salmon',
 4: 'hamburger',
 5: 'ice_cream',
 6: 'pizza',
 7: 'ramen',
 8: 'steak',
 9: 'sushi'}

In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [9]:
from torchvision import transforms as T

In [10]:
transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
from torchvision.datasets import ImageFolder

In [12]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=transform
)

In [13]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 750
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [14]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=transform
)

In [15]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2500
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [16]:
from torch.utils.data import DataLoader

In [17]:
BATCH_SIZE = 16

In [18]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [19]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [20]:
len(train_loader), len(test_loader)

(47, 157)

In [21]:
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class FoodNet(nn.Module):
    def __init__(self):
        super(FoodNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 8, 3)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.fc1 = nn.Linear(BATCH_SIZE * 26 * 26, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, BATCH_SIZE * 26 * 26)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [23]:
model = FoodNet()
model = model.to(device)

In [24]:
import torch.optim as optim

In [25]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [26]:
EPOCHS = 20

In [27]:
loaders = {
    'train': train_loader,
    'val': test_loader
}

In [28]:
for epoch in range(1, EPOCHS+1):
    running_loss = .0
    running_acc = .0
    for phase in ['train', 'val']:
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, targets in loaders[phase]:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            accuracy = (torch.argmax(outputs, dim=1) == targets).float().mean()

            running_loss += loss.item()
            running_acc += accuracy.item()
        print(f"{phase.upper()} Epoch {epoch}/{EPOCHS}, Loss: {running_loss/len(loaders[phase])}, Acc: {running_acc/len(loaders[phase])}")

TRAIN Epoch 1/20, Loss: 2.304821308622969, Acc: 0.09574468085106383
VAL Epoch 1/20, Loss: 2.986589039966559, Acc: 0.13415605095541402
TRAIN Epoch 2/20, Loss: 2.2846321907449276, Acc: 0.14779635265152505
VAL Epoch 2/20, Loss: 2.9548803848825442, Acc: 0.18437215652625272
TRAIN Epoch 3/20, Loss: 2.229900933326559, Acc: 0.17762158081886617
VAL Epoch 3/20, Loss: 2.890502058017026, Acc: 0.2235555050859026
TRAIN Epoch 4/20, Loss: 2.1561252386011978, Acc: 0.1905395138136884
VAL Epoch 4/20, Loss: 2.852905078298727, Acc: 0.2254322111416774
TRAIN Epoch 5/20, Loss: 2.0879703912329166, Acc: 0.22568389075867673
VAL Epoch 5/20, Loss: 2.814698149444191, Acc: 0.2578480437303045
TRAIN Epoch 6/20, Loss: 2.028721033258641, Acc: 0.2841945290565491
VAL Epoch 6/20, Loss: 2.7945629031794845, Acc: 0.28332575073667393
TRAIN Epoch 7/20, Loss: 1.97694933668096, Acc: 0.29692249253709263
VAL Epoch 7/20, Loss: 2.784767879042656, Acc: 0.28474749776588126
TRAIN Epoch 8/20, Loss: 1.9028901216831613, Acc: 0.366071429658